In [1]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 59 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 2.3 MB/s 


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.rfr ,2.xgbr, 3.ann)

In [3]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc.h5')

#pickle for rfr
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
#    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_tikamaveragegcr5-5r6.h5')
# Show the model architecture
#new_model.summary()

In [4]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [5]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [6]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [7]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [8]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [9]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [10]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

[[2.0e+00 3.1e-01 7.5e-01 5.0e+02]
 [2.0e+00 4.2e-01 3.0e+00 7.0e+02]
 [2.0e+00 2.5e-01 7.5e-01 5.0e+02]
 [2.0e+00 3.2e-01 7.5e-01 2.0e+02]
 [1.0e+00 3.5e-01 1.0e+00 1.0e+02]
 [1.0e+00 5.4e-01 1.0e+00 8.0e+02]
 [1.0e+00 5.2e-01 3.0e+00 4.0e+02]
 [1.0e+00 4.9e-01 3.0e+00 3.0e+02]
 [2.0e+00 5.2e-01 7.5e-01 9.0e+02]
 [1.0e+00 3.2e-01 1.0e+00 7.0e+02]
 [1.0e+00 4.8e-01 1.0e+00 8.0e+02]
 [2.0e+00 2.8e-01 3.0e+00 4.0e+02]
 [2.0e+00 4.7e-01 3.0e+00 6.0e+02]
 [1.0e+00 3.8e-01 7.5e-01 1.0e+02]
 [2.0e+00 3.1e-01 7.5e-01 5.0e+02]
 [1.0e+00 4.5e-01 7.5e-01 7.0e+02]
 [1.0e+00 5.7e-01 1.0e+00 1.0e+02]
 [1.0e+00 3.9e-01 1.0e+00 3.0e+02]
 [1.0e+00 3.9e-01 7.5e-01 8.0e+02]
 [2.0e+00 4.6e-01 3.0e+00 6.0e+02]]
[24.183208  34.97407    7.6136107 26.44489   12.776992  33.801144
 35.057617  28.906303  32.388855  42.2551    33.801144  11.359172
 33.397793  10.730387  24.183208  38.81412   13.617972  34.209133
 39.871304  38.827038 ]


In [11]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 4].argsort()]
    parents_pool=mixsort[:,0:4]
    parents_results=mixsort[:,4]
    return parents_pool

In [12]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


#crossover_pool=crossover_process(parents_pool)
#print(crossover_pool)



In [13]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [14]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:4]
    return new_mating_pool


test2=survivor_selection()
print(test2)

[[1.00000000e+00 3.20000000e-01 1.00000000e+00 7.00000000e+02]
 [1.00000000e+00 3.90000000e-01 7.50000000e-01 8.00000000e+02]
 [2.00000000e+00 4.60000000e-01 3.00000000e+00 6.00000000e+02]
 [1.00000000e+00 4.50000000e-01 7.50000000e-01 7.00000000e+02]
 [1.00000000e+00 5.20000000e-01 3.00000000e+00 4.00000000e+02]
 [2.00000000e+00 4.20000000e-01 3.00000000e+00 7.00000000e+02]
 [1.00000000e+00 3.90000000e-01 1.00000000e+00 3.00000000e+02]
 [1.00000000e+00 5.40000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.80000000e-01 1.00000000e+00 8.00000000e+02]
 [2.00000000e+00 4.70000000e-01 3.00000000e+00 6.00000000e+02]
 [2.00000000e+00 5.20000000e-01 7.50000000e-01 9.00000000e+02]
 [1.00000000e+00 3.20793687e-01 2.53258174e+00 3.37677565e+02]
 [1.00000000e+00 4.90000000e-01 3.00000000e+00 3.00000000e+02]
 [2.00000000e+00 3.20000000e-01 7.50000000e-01 2.00000000e+02]
 [2.00000000e+00 3.10000000e-01 7.50000000e-01 5.00000000e+02]
 [2.00000000e+00 3.10000000e-01 7.50000000e-01 5.000000

In [15]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


[[1.00000000e+00 3.20000000e-01 1.00000000e+00 7.00000000e+02]
 [1.00000000e+00 3.90000000e-01 7.50000000e-01 8.00000000e+02]
 [2.00000000e+00 4.60000000e-01 3.00000000e+00 6.00000000e+02]
 [1.00000000e+00 4.50000000e-01 7.50000000e-01 7.00000000e+02]
 [1.00000000e+00 5.20000000e-01 3.00000000e+00 4.00000000e+02]
 [2.00000000e+00 4.20000000e-01 3.00000000e+00 7.00000000e+02]
 [1.00000000e+00 3.90000000e-01 1.00000000e+00 3.00000000e+02]
 [1.00000000e+00 5.40000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.80000000e-01 1.00000000e+00 8.00000000e+02]
 [2.00000000e+00 4.70000000e-01 3.00000000e+00 6.00000000e+02]
 [2.00000000e+00 5.20000000e-01 7.50000000e-01 9.00000000e+02]
 [1.00000000e+00 4.90000000e-01 3.00000000e+00 3.00000000e+02]
 [2.00000000e+00 3.20000000e-01 7.50000000e-01 2.00000000e+02]
 [2.00000000e+00 3.10000000e-01 7.50000000e-01 5.00000000e+02]
 [2.00000000e+00 3.10000000e-01 7.50000000e-01 5.00000000e+02]
 [1.00000000e+00 3.43408215e-01 2.71111749e+00 9.037058

In [16]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

Streaming output truncated to the last 5000 lines.
 [1.00000000e+00 3.56544550e-01 3.30499471e+00 8.06399095e+02]
 [1.00000000e+00 3.56221358e-01 2.86822469e+00 7.82563976e+02]
 [1.00000000e+00 3.57743532e-01 2.85258890e+00 8.07932160e+02]
 [1.00000000e+00 3.57344563e-01 1.09690053e+00 7.21400305e+02]
 [1.00000000e+00 3.57411820e-01 1.09696417e+00 7.21442161e+02]
 [1.00000000e+00 3.56632390e-01 1.09457196e+00 7.22115585e+02]
 [1.00000000e+00 3.57365296e-01 1.09863487e+00 7.21834035e+02]]
best results: [12.05971   24.648563  47.781242   1.6486979 15.027725 ]
Fittest C2H4+ethanol: 62.808968
[array([2.00000000e+00, 3.58703703e-01, 5.11409529e-01, 8.18821490e+02])]
gen 784 = [[2.00000000e+00 3.58703703e-01 5.11409529e-01 8.18821490e+02]
 [2.00000000e+00 3.56220065e-01 5.10679726e-01 7.61324334e+02]
 [2.00000000e+00 3.58191817e-01 5.26208028e-01 7.84474018e+02]
 [2.00000000e+00 3.56729133e-01 5.49247561e-01 7.85595096e+02]
 [2.00000000e+00 3.57253106e-01 5.80088253e-01 8.10511073e+02]
 [1.0

In [17]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))


[12.05971   24.648563  47.781242   1.6486979 15.027725 ] [2.00000000e+00 3.58703703e-01 5.11409529e-01 8.18821490e+02]
[[12.05971   24.648563  47.781242   1.6486979 15.027725 ]
 [12.05971   24.648563  47.781242   1.6486979 15.027725 ]
 [12.05971   24.648563  47.781242   1.6486979 15.027725 ]
 [12.05971   24.648563  47.781242   1.6486979 15.027725 ]
 [12.05971   24.648563  47.781242   1.6486979 15.027725 ]
 [18.24443   24.875105  47.758778   1.7104616 14.479974 ]
 [18.24443   24.875105  47.758778   1.7104616 14.479974 ]
 [18.24443   24.875105  47.758778   1.7104616 14.479974 ]
 [16.076729  22.460752  47.282627   2.68045   16.126547 ]
 [16.076729  22.460752  47.282627   2.68045   16.126547 ]
 [17.011063  23.962683  47.225582   1.6796112 14.627724 ]
 [17.011063  23.962683  47.225582   1.6796112 14.627724 ]
 [17.011063  23.962683  47.225582   1.6796112 14.627724 ]
 [16.002357  22.395584  47.120407   1.6234972 15.638962 ]
 [16.002357  22.395584  47.120407   1.6234972 15.638962 ]
 [16.002357